In [31]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import datetime
import time

In [32]:
taglist = ['avocado','avocados','avocadotoast','wherearetheavocados','avocadooil','avocadolove','avocadolover','avocadoplant','avocadosalad','avocadoporn','avocadotree','avocadolovers','avocadoaddict','aguacate']

In [33]:
driver = webdriver.Chrome('/Users/adriaferrer/IronhackRepos/Avocalypse/Tests/chromedriver')

In [34]:
# Define dataframe to store hashtag information
tag_df  = pd.DataFrame(columns = ['Hashtag', 'Number of Posts', 'Posting Freq (mins)'])
 
# Loop over each hashtag to extract information
for tag in taglist:
     
    driver.get('https://www.instagram.com/explore/tags/'+str(tag))
    soup = BeautifulSoup(driver.page_source,"lxml")
     
    # Extract current hashtag name
    tagname = tag
    # Extract total number of posts in this hashtag
    # NOTE: Class name may change in the website code
    # Get the latest class name by inspecting web code
    nposts = soup.find('span', {'class': 'g47SY'}).text
         
    # Extract all post links from 'explore tags' page
    # Needed to extract post frequency of recent posts
    myli = []
    for a in soup.find_all('a', href=True):
        myli.append(a['href'])
 
    # Keep link of only 1st and 9th most recent post 
    newmyli = [x for x in myli if x.startswith('/p/')]
    del newmyli[:9]
    del newmyli[9:]
    del newmyli[1:8]
 
    timediff = []
 
    # Extract the posting time of 1st and 9th most recent post for a tag
    for j in range(len(newmyli)):
        driver.get('https://www.instagram.com'+str(newmyli[j]))
        soup = BeautifulSoup(driver.page_source,"lxml")
 
        for i in soup.findAll('time'):
            if i.has_attr('datetime'):
                timediff.append(i['datetime'])
                #print(i['datetime'])
 
    # Calculate time difference between posts
    # For obtaining posting frequency
    datetimeFormat = '%Y-%m-%dT%H:%M:%S.%fZ'
    diff = datetime.datetime.strptime(timediff[0], datetimeFormat)\
        - datetime.datetime.strptime(timediff[1], datetimeFormat)
    pfreq= int(diff.total_seconds()/(9*60))
     
    # Add hashtag info to dataframe
    tag_df.loc[len(tag_df)] = [tagname, nposts, pfreq]
         
driver.quit()

In [35]:
# Check the final dataframe
print(tag_df)
 
# CSV output for hashtag analysis
tag_df.to_csv('hashtag_list.csv')

                Hashtag Number of Posts Posting Freq (mins)
0               avocado       9.468.466                   0
1              avocados         404.676                   1
2          avocadotoast       1.139.201                   1
3   wherearetheavocados         482.684                   0
4            avocadooil         112.418                   7
5           avocadolove         246.162                   2
6          avocadolover         185.639                   1
7          avocadoplant          28.850                  16
8          avocadosalad          76.606                   5
9           avocadoporn          24.581                  34
10          avocadotree          55.272                  20
11        avocadolovers          49.061                  11
12        avocadoaddict          64.657                  10
13             aguacate         586.821                   1
